#### Giovanni Gamaliel López Padilla
#### Procesamiento de lenguaje natural
#### Tarea 02

In [1]:
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_recall_fscore_support, roc_auc_score
from sklearn.model_selection import GridSearchCV
from keras.preprocessing.text import Tokenizer
from nltk.tokenize import TweetTokenizer
from collections import Counter
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn import svm
import numpy as np
import nltk
import os
import re

In [67]:
def join_path(path: str, filename: str) -> str:
    """
    Une la direccion de un archivo con su nombre
    """
    return "{}{}".format(path, filename)


def get_texts_from_file(path_data: str, path_labels: str) -> tuple:
    """
    Obtiene una lista de oraciones a partir de un texto con sus respectivas etiquetas
    """
    # Inicilizacion de las listas
    text = []
    labels = []
    # Apertura de los archivos
    with open(path_data, "r") as f_data, open(path_labels, "r") as f_labels:
        # Recoleccion de las oraciones
        for tweet in f_data:
            text += [tweet]
        # Recoleccion de las etiquedas
        for label in f_labels:
            labels += [label]
    # Etiquedas a enteros
    labels = list(map(int, labels))
    return text, labels


def sort_freqdist(fdist: nltk.FreqDist) -> list:
    """
    Ordena la lista de distribucion de frecuencias de palabras de mayor frecuencia a menor
    """
    aux = [(fdist[key], key) for key in fdist]
    aux.sort()
    aux.reverse()
    return aux


def split_data(data: list, max_words: int) -> list:
    """
    Realiza la separacion de elementos en una lista dado el numero de elementos que se quieren conservar
    """
    return data[:max_words]


def obtain_fdist(data: list, max_words: int) -> list:
    """
    Obtiene la lista de una distribucion de frecuencias de palabras ordenada de mayor a menor a partir de una lista de oraciones
    """
    # Inicializacion del Tokenizador
    tokenizer = TweetTokenizer()
    # Inicializacion de la lista que guardara los tokens
    corpus_palabras = []
    for tweet in data:
        # Creacion y guardado de los tokens
        corpus_palabras += tokenizer.tokenize(tweet)
    # Creacion de la distribucion de frecuencias
    fdist = nltk.FreqDist(corpus_palabras)
    fdist = sort_freqdist(fdist)
    fdist = split_data(fdist, max_words)
    return fdist


def create_dictonary_of_index(fdist: list) -> dict:
    """
    Crea un diccionario con la posición de mayor a menor frecuencia de cada palabra. La llave es la palabra a consultar
    """
    # Inicializacion del diccionario
    index = dict()
    # Inicializacion de la posicion
    i = 0
    for weight, word in fdist:
        index[word] = i
        i += 1
    return index


def build_binary_bow(data: list, fdist: list, index: dict) -> np.array:
    """
    Creacion de la BoW usando pesos binarios
    """
    tokenizer = TweetTokenizer()
    bow = np.zeros((len(data), len(fdist)), dtype=float)
    docs = 0
    for tweet in data:
        fdist_data = nltk.FreqDist(tokenizer.tokenize(tweet))
        for word in fdist_data:
            if word in index.keys():
                bow[docs, index[word]] = 1
        docs += 1
    return bow


def build_frecuency_bow(data: list, fdist: list, index: dict) -> np.array:
    """
    Creacion de la BoW usando pesos basado en frecuencias
    """
    tokenizer = TweetTokenizer()
    bow = np.zeros((len(data), len(fdist)), dtype=float)
    docs = 0
    for tweet in data:
        fdist_data = nltk.FreqDist(tokenizer.tokenize(tweet))
        for word in fdist_data:
            if word in index.keys():
                bow[docs, index[word]] = tweet.count(word)
        docs += 1
    return bow


def create_empty_dictionary_of_words_and_documents(words: dict,
                                                   data: list) -> dict:
    # Inicializacion del diccionario
    freq_word_per_document = dict()
    word_count = dict()
    for i, tweet in enumerate(data):
        word_count[i] = 0
    for word in words:
        freq_word_per_document[word] = word_count
    return freq_word_per_document

def build_tfidf_bow(data: list, fdist: list, index: dict) -> np.array:
    """
    Creacion de la BoW usando pesos basado en frecuencias
    """
    tokenizer = TweetTokenizer()
    # Inicilizacion del bow
    bow = np.zeros((len(data), len(fdist)), dtype=float)
    # Total de oraciones
    n = len(data)
    # Inicializacion del diccionario que contiene la repeticion de cada palabra
    idf_per_word_and_document = create_empty_dictionary_of_words_and_documents(
        index.keys(), data)
    for docs, tweet in enumerate(data):
        # Frecuencias 
        fdist_data = nltk.FreqDist(tokenizer.tokenize(tweet))
        for word in fdist_data:
            if word in index.keys():
                # Descriptiva
                tf = tweet.count(word)
                idf_per_word_and_document[word][docs] += 1
                bow[docs, index[word]] = tf
    # Discriminativa
    for word in index.keys():
        idf = sum(idf_per_word_and_document[word].values())
        idf = np.log(n / idf)
        for docs, tweet in enumerate(data):
            bow[docs, index[word]] = bow[docs, index[word]] * idf
    return bow


def create_model(bow_tr: np.array, labels_tr: np.array) -> GridSearchCV:
    """
    Creacion del modelo para realizar el aprendizaje
    """
    parameters_model = {"C": [0.05, 0.12, 0.25, 0.5, 1, 2, 4]}
    svr = svm.LinearSVC(class_weight="balanced", max_iter=1200000)
    grid = GridSearchCV(estimator=svr,
                        param_grid=parameters_model,
                        n_jobs=8,
                        scoring="f1_macro",
                        cv=5)
    grid.fit(bow_tr, labels_tr)
    return grid


def evaluate_model(bow_val: np.array, labels_val: np.array,
                   grid: GridSearchCV) -> np.array:
    """
    Resultados del modelo con el dataset de validacion
    """
    y_pred = grid.predict(bow_val)
    p, r, f, _ = precision_recall_fscore_support(labels_val,
                                                 y_pred,
                                                 average="macro",
                                                 pos_label=1)
    print(confusion_matrix(labels_val, y_pred))
    print(metrics.classification_report(labels_val, y_pred))
    return y_pred

In [38]:
parameters = {"path data": "../Data/",
              "train": {"data": "mex_train.txt",
                        "labels": "mex_train_labels.txt"},
              "validation": {"data": "mex_val.txt",
                             "labels": "mex_val_labels.txt"},
              "max words": 5000,
              }
# Definicion de las rutas de cada archivo de datos y validacion
path_data_tr = join_path(parameters["path data"],
                         parameters["train"]["data"])
path_label_tr = join_path(parameters["path data"],
                          parameters["train"]["labels"])
path_data_val = join_path(parameters["path data"],
                          parameters["validation"]["data"])
path_label_val = join_path(parameters["path data"],
                           parameters["validation"]["labels"])
# Lectura de las oraciones y etiquetas de los datos de entrenamiento y validacion
data_tr, labels_tr = get_texts_from_file(path_data_tr,
                                         path_label_tr)
data_val, labels_val = get_texts_from_file(path_data_val,
                                           path_label_val)

In [39]:
# Obtiene la distribucion de palabras ordenadas de mayor a menor con un maximo de 5000 palabras
fdist_tr = obtain_fdist(data_tr,
                       parameters["max words"])
# Creacion del diccionario con la posicion en la distribucion de palabras
word_index = create_dictonary_of_index(fdist_tr)

In [69]:
# Creacion de la BoW para los datos de entrenamiento usando pesos binarios
binary_bow_tr = build_binary_bow(data_tr,
                          fdist_tr,
                          word_index)
# Creacion de la BoW para los datos de validacion usando pesos binarios
binary_bow_val = build_binary_bow(data_val,
                          fdist_tr,
                          word_index)

In [70]:
grid=create_model(binary_bow_tr,labels_tr)
y_pred = evaluate_model(binary_bow_val,labels_val,grid)

[[329  68]
 [ 47 172]]
              precision    recall  f1-score   support

           0       0.88      0.83      0.85       397
           1       0.72      0.79      0.75       219

    accuracy                           0.81       616
   macro avg       0.80      0.81      0.80       616
weighted avg       0.82      0.81      0.82       616



In [9]:
# Creacion de la BoW para los datos de entrenamiento usando pesos binarios
freq_bow_tr = build_frecuency_bow(data_tr, fdist_tr, word_index)
# Creacion de la BoW para los datos de validacion usando pesos binarios
freq_bow_val = build_frecuency_bow(data_val, fdist_tr, word_index)

In [10]:
grid=create_model(freq_bow_tr,labels_tr)
y_pred = evaluate_model(freq_bow_val,labels_val,grid)

[[333  64]
 [ 49 170]]
              precision    recall  f1-score   support

           0       0.87      0.84      0.85       397
           1       0.73      0.78      0.75       219

    accuracy                           0.82       616
   macro avg       0.80      0.81      0.80       616
weighted avg       0.82      0.82      0.82       616



In [55]:
# Creacion de la BoW para los datos de entrenamiento usando pesos binarios
tfidf_bow_tr = build_tfidf_bow(data_tr, fdist_tr, word_index)
# Creacion de la BoW para los datos de validacion usando pesos binarios
tfidf_bow_val = build_tfidf_bow(data_val, fdist_tr, word_index)

In [68]:
grid=create_model(tfidf_bow_tr,labels_tr)
y_pred = evaluate_model(tfidf_bow_val,labels_val,grid)

[[327  70]
 [ 62 157]]
              precision    recall  f1-score   support

           0       0.84      0.82      0.83       397
           1       0.69      0.72      0.70       219

    accuracy                           0.79       616
   macro avg       0.77      0.77      0.77       616
weighted avg       0.79      0.79      0.79       616

